In [2]:
#!/usr/bin/env python
# coding: utf-8

import functools
import nest_asyncio
import os
import traceback
import pandas as pd
import telebot
from icecream import ic
from IPython import get_ipython
from IPython.utils.capture import capture_output
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from typing import TypedDict, Annotated, Sequence
import operator
from langchain import hub
from gsheet_handler import GSheetHandler
import importlib
import json

nest_asyncio.apply()

# Импорт и обновление модуля gsheet_handler
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

# Извлечение данных из Google Sheets
example_dialog = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1sBAFF62fAjSdPrGYTWgpjmBjLRiFCb8DNAX8ZypI-_U/edit?gid=0#gid=0', 0)
print(example_dialog)

# Функция создания тренировочного промпта
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage

def create_training_prompt(dialogs):
    instruction_message = SystemMessage(content="Ты консультант по инвестиционным программам правительства Москвы.")
    examples_message_content = "Примеры диалогов:\n"
    
    for i, row in dialogs.iterrows():
        examples_message_content += f"Человек: {row['Human']}\nAI: {row['AI']}\n"

    examples_message = SystemMessage(content=examples_message_content)
    
    messages = [instruction_message, examples_message]
    prompt_template = ChatPromptTemplate.from_messages(messages)
    return prompt_template

# Пример использования функции с `example_dialog`
training_prompt = create_training_prompt(example_dialog)
formatted_messages = training_prompt.format_messages()

for message in formatted_messages:
    print(f"{message.type}: {message.content}")

# Загрузка конфигурации
with open('config.json') as config_file:
    config = json.load(config_file)

os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "nlmk_bot"
TELEGRAMM_API_KEY = config["TELEGRAM_BOT_TOKEN"]

# Создание бота
bot = telebot.TeleBot(TELEGRAMM_API_KEY)

# Инициализация модели ChatOpenAI
chat = ChatOpenAI()

# История сообщений
message_history = training_prompt.format_messages()

# Функция создания промпта с историей
def create_prompt_with_history(history, user_input):
    messages = history + [HumanMessage(content=user_input)]
    return ChatPromptTemplate.from_messages(messages)

# Функция обработки сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    ic(f"Received message: {message.text}")

    # Проверка, направлено ли сообщение боту
    if message.reply_to_message and message.reply_to_message.from_user.id == bot.get_me().id:
        bot_called = True
    elif f"@{bot.get_me().username}" in message.text:
        bot_called = True
    else:
        bot_called = False

    if bot_called:
        try:
            # Создание промпта с историей и вводом пользователя
            prompt_with_history = create_prompt_with_history(message_history, message.text)
            formatted_messages = prompt_with_history.format_messages(user_input=message.text)
            response = chat.invoke(formatted_messages)
            ic(f"Generated response: {response.content}")
            
            # Добавление нового сообщения пользователя и ответа AI в историю
            message_history.append(HumanMessage(content=message.text))
            message_history.append(AIMessage(content=response.content))
            
            bot.reply_to(message, response.content)
        except Exception as e:
            error_message = f"Произошла ошибка: {e}"
            bot.reply_to(message, error_message)
            ic(error_message)
            traceback.print_exc()

# Запуск бота
if __name__ == "__main__":
    ic("Starting bot polling")
    bot.polling(none_stop=True)


no docx


/home/fedor/miniconda3/envs/jupyterlab/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
ic| 'Starting bot polling'


                                                Human  \
0   Интересует программа «Земля за 1 рубль для про...   
1   Где найти более подробную информацию о конкрет...   
2         Как подать заявку на данную меру поддержки?   
3   Интересует возможность вступления в реестр аль...   
4   Хочу узнать информацию о предоставлении столич...   
5   Как подать заявку на присвоение статуса промыш...   
6   Необходима помощь в поиске площадки в Москве д...   
7                           Интересуют льготные займы   
8       Интересует программа компенсации по кредитам    
9   Как зарегистрироваться на Инвестиционном портале?   
10          Как найти интересующий объект на портале?   
11                          Как подписаться на торги?   
12                              Навигация по порталу    
13                     Разъяснения по онлайн-сервисам   
14  Как выставить на торги интересующий объект нед...   
15   Как направить предложение в Московский инвестор?   
16  Что делать, если моей пробл

ic| f"Received message: {message.text}": 'Received message: кто-кто?'
ic| f"Generated response: {response.content}": ('Generated response: Я - искусственный интеллект, который специализируется на '
                                                'консультациях по инвестиционным программам правительства Москвы. Если у вас '
                                                'есть вопросы или нужна помощь в этой области, не стесняйтесь обращаться ко '
                                                'мне.')
